# Differential Methylation with LIMMA

## Initialisation

In [ ]:
import glob
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


### Parameters and File Paths

In [ ]:
#--- Parameters
REGIONS = 'hg19_cpg_clusters_k3_s150_w150'
FILTER_COV = 10
#--- Local paths
ROOT_DIR = '/home/ubuntu/git/etsang/projects'
PROJECT_SLUG = '2023_10_10_SRT_hyper_tissue_dmr_selection_EKT'
PROJECT_DIR = f"{ROOT_DIR}/{PROJECT_SLUG}/work"

ALAN_PROJECT_DIR = "/home/ubuntu/data/2023_07_10_HYPER_design_AS"
# Samples
SAMPLE_PATH = ALAN_PROJECT_DIR + '/stage/metadata/loyfer2022_samples_with_blueprint.tsv'

# Methylation data
METH_DIR = (
    PROJECT_DIR + 
    '/bp_loyfer_meth_summaries/standard-{regions}.filtered'
).format(regions=REGIONS)
# Where to store the results
RESULTS_PATH = (
    PROJECT_DIR + 
    '/diff_meth/standard_diffmeth_{regions}_%s_minus_%s.tsv.gz'
).format(regions=REGIONS)

### Sample Metadata

In [ ]:
samples_df = pd.read_csv(SAMPLE_PATH, sep='\t')
ridxs = ~(samples_df['super_group'].isna() | samples_df['super_group'].str.startswith('Blueprint-'))
# drop umbilical endothelium
ridxs = ridxs & (samples_df['super_group'] != 'Umbilical-Endothelium')
samples_df = samples_df[ridxs].copy()
samples_df['sample_group'] = samples_df['super_group']\
    .str.replace('-', '_', regex=False)\
    .str.replace('+', '_plus_', regex=False)

In [ ]:
summary = samples_df\
    .groupby('sample_group')\
    .size()
summary

In [ ]:
samples_df.shape[0], summary.sum()

### Methylation Data

In [ ]:
%%time
METH_COLS = [
    'sample_id', 'region_id', 'region_number_total', 'region_meth_rate'
]
meth_df = pd.concat([
    pd.read_csv(ifile, sep='\t', names=METH_COLS)
    for ifile in glob.glob(METH_DIR+'/*.csv')
])

## Align Methylation Data and Sample/Probe Metadata

In [ ]:
%%time
ridxs = meth_df['sample_id'].isin(samples_df['sample_id'])
ridxs &= (meth_df['region_number_total']>=FILTER_COV)
meth_df = meth_df[ridxs]\
    .pivot_table(index='region_id', columns='sample_id', values='region_meth_rate')
meth_df.shape

In [ ]:
# Remove samples with many missing values
cidxs = (meth_df.isna().sum(axis=0)<(0.05*meth_df.shape[0]))
valid_meth_df = meth_df.loc[:, cidxs].copy()
# Remove missing values
ridxs = (valid_meth_df.isna().sum(axis=1)==0)
valid_meth_df = valid_meth_df.loc[ridxs, :].copy()
valid_meth_df.shape

In [ ]:
# align with sample metadata
ridxs = samples_df['sample_id'].isin(valid_meth_df.columns)
meta_df = samples_df[ridxs][['sample_id', 'sample_group']].copy()
valid_meth_df = valid_meth_df.loc[:, meta_df['sample_id']].copy()
SAMPLE_GROUPS = sorted(meta_df['sample_group'].unique())
valid_meth_df.shape, len(SAMPLE_GROUPS)

In [ ]:
ridxs = ~samples_df['sample_id'].isin(meta_df['sample_id'])
summary = samples_df[ridxs]\
    .groupby('sample_group')\
    .size()
summary

## Differential Methylation Analysis

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R -i valid_meth_df,meta_df,SAMPLE_GROUPS,RESULTS_PATH

library(limma)
library(minfi)
library(tidyverse)

B <- as.matrix(valid_meth_df)
M <- log2(B) - log2(1-B)
group = factor(meta_df[['sample_group']], levels=SAMPLE_GROUPS)

################################################
#--- LIMMA MODELING
################################################
arrays <- ExpressionSet(assayData = M)
design <- model.matrix(~ 0 + group)
colnames(design) <- levels(group)
fit <- lmFit(arrays, design)

#--- All pairwise comparisons
CELL_TYPES = levels(group)
for (i in 1:(length(CELL_TYPES)-1)) {
    ct1 = CELL_TYPES[i]
    for (j in (i+1):length(CELL_TYPES)) {
        ct2 = CELL_TYPES[j]
        contrast = sprintf("%s - %s", ct2, ct1)
        print(sprintf("--> %s", contrast))
        contrasts <- makeContrasts(contrast, levels = design)
        fit2 <- contrasts.fit(fit, contrasts)
        fit2 <- eBayes(fit2)
        # Diff meth stats
        tt <- topTable(fit2, coef = contrast, 
                       adjust="BH", number=nrow(arrays), sort.by = "none")
        M1 <- fit$coefficients[, ct1]
        B1 <- 2**M1/(1+2**M1)
        M2 <- fit$coefficients[, ct2]
        B2 <- 2**M2/(1+2**M2)
        rv <- data.frame(
            region_id=row.names(valid_meth_df),
            logfc=tt$logFC,
            tstat=tt$t,
            pval=tt$P.Value,
            fdr=tt$adj.P.Val,
            meth_base=B1,
            meth_delta=B2-B1
        )
        # Filter results to reduce the output size
        FILTER_FDR <- 0.05
        FILTER_DELTA <- 0.2
        ofile <- sprintf(RESULTS_PATH, ct2, ct1)
        rv <- as_tibble(rv) %>%
            filter(fdr<=FILTER_FDR, abs(meth_delta)>=FILTER_DELTA)
        write_tsv(rv, file=ofile, progress=FALSE)
    }
}
 